In [1]:
from writers import foyer_xml_writer
from writers.foyer_xml_writer import parmed_to_foyer_xml, mbuild_to_foyer_xml


import ele
import espaloma as esp
import forcefield_utilities as ffutils
import foyer
import gmso
import mbuild as mb
from mbuild.lib.recipes import Polymer
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield
import numpy as np
from openff.toolkit.topology import Molecule
import torch
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield
import hoomd
import gsd.hoomd
import matplotlib.pyplot as plt

import os
import warnings
warnings.filterwarnings("ignore")

if not os.path.exists("espaloma_model.pt"):
    os.system("wget http://data.wangyq.net/espaloma_model.pt")

/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/qcelemental/models/results.py:788: DeprecationWarning: ResultProtocols has been renamed to AtomicResultProtocols and will be removed as soon as v0.13.0
  warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/qcelemental/models/results.py:788: DeprecationWarning: ResultProtocols has been renamed to AtomicResultProtocols and will be removed as soon as v0.13.0
  warn(
LICENSE: Could not open license file "oe_license.txt" in local directory
LICENSE: N.B. OE_LICENSE environment variable is not set
LICENSE: N.B. OE_DIR environment variable is not set
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!


In [11]:
mb.load("C1([C]2[C](S[CH][CH]2)[C]2[C]1[CH][C](S2)[C]1[CH]=N[CH]=[C]2[C]1NSN2)(CCCCCCCCCCCCCCCC)CCCCCCCCCCCCCCCC",smiles=True).visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [12]:
#cant get the sdf file or mol file to work with N-S-N bonds, it works from smiles string though...
#either need to get smiles string directly from mol2 file or figure out how to get the 
#smiles string from the smi file into a usable format. 
#it worked to run openbabel and get the smile string from the smi file and then just 
#delete the brackets around the N's.

#molecule = Molecule.from_file("cpdt_test.sdf",file_format = "sdf")

molecule = Molecule.from_smiles("C1([C]2[C](S[CH][CH]2)[C]2[C]1[CH][C](S2)[C]1[CH]=N[CH]=[C]2[C]1NSN2)(CCCCCCCCCCCCCCCC)CCCCCCCCCCCCCCCC")
molecule_graph = esp.Graph(molecule)
#either generate smiles string from the built fragments or get our fragments into a Molecule supported file format

espaloma_model = torch.load("espaloma_model.pt")
espaloma_model(molecule_graph.heterograph)
openmm_system = esp.graphs.deploy.openmm_system_from_graph(molecule_graph)

# Store the results for each in something more accessible
pair_forces = openmm_system.getForces()[1]
angle_forces = openmm_system.getForces()[3]
bond_forces = openmm_system.getForces()[2]
torsion_forces = openmm_system.getForces()[0]